In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from collections import Counter

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    
    recs = np.random.choice(items_weights['item_id'].tolist(), n, p=items_weights['weight'].tolist())
    
    return recs

In [6]:
%%time

sales_value_sum = data_train['sales_value'].sum()
weighted = data_train.groupby('item_id')['sales_value'].sum().reset_index()
weighted.rename(columns={'sales_value': 'weight'}, inplace=True)
weighted['weight'] = weighted['weight'].apply(lambda x: x/sales_value_sum)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weighted, n=5))
result.head(10)

Wall time: 46 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[10284976, 6534178, 5568789, 5568231, 1036495]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[826249, 962782, 13008133, 7092683, 133258]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1060269, 961554, 6534178, 1096315, 960421]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1127227, 1131449, 1089074, 1029743, 5565187]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1062002, 945353, 5576740, 1067795, 10355878]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[983267, 919948, 1004705, 9297020, 1041487]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[1089393, 983699, 1106523, 866211, 911703]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[2675586, 1065538, 1060327, 841128, 1080014]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[950550, 1066776, 1017220, 921504, 1118235]"
9,16,"[1062973, 1082185, 13007710]","[9836835, 10457112, 995242, 1122298, 899624]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [7]:
top_5000 = np.random.choice(data_train['item_id'].tolist(), size=5000, replace=False)
top_5000

array([ 977166,  946788, 1005186, ..., 1041259, 1076444, 1130838])

In [8]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [9]:
data_train.head(100)
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

C:\Users\zhunt\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,6666,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,6666,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [10]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


In [11]:
user_item_matrix.shape

(2499, 3552)

In [12]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

4.630467051685539

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [14]:
%%time

model = ItemItemRecommender(K=25, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 941 ms


In [15]:
recs

[(0, 62798.0),
 (2177, 57368.0),
 (1398, 48105.0),
 (2303, 42228.0),
 (1498, 41608.0)]

In [16]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 1098066, 995242]

In [17]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[user_id], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 173 ms


In [18]:
result.head(5)

,user_id,actual,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[10284976, 6534178, 5568789, 5568231, 1036495]","[6666, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[826249, 962782, 13008133, 7092683, 133258]","[6666, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1060269, 961554, 6534178, 1096315, 960421]","[6666, 1082185, 981760, 995242, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1127227, 1131449, 1089074, 1029743, 5565187]","[6666, 1082185, 981760, 995242, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1062002, 945353, 5576740, 1067795, 10355878]","[6666, 1082185, 981760, 995242, 1098066]"


In [22]:
# Функции из 1-ого вебинара
import os, sys
    
from metrics import precision_at_k, recall_at_k

In [23]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0214:weighted_random_recommendation
0.1504:itemitem
